Neuron Lang is a python based DSL for naming neurons. Neurons are modelled as collections of phenotypes with semantics backed by Web Ontology Language (OWL2) classes. Neuron Lang provides tools for mapping to and from collections of local names for phenotypes by using ontology identifiers as the common language underlying all local naming. These tools also let us automatically generate names for neurons in a regular and consistent way using a set of rules operating on the neurons' constitutent phenotypes. Neuron Lang can export to python or to any serialziation supported by rdflib, however [deterministic turtle](https://github.com/tgbugs/pyontutils/blob/master/docs/ttlser.md) (ttl) is prefered. Neuron Lang depends on files from the [NIF-Ontology](https://github.com/SciCrunch/NIF-Ontology).

This notebook has examples of how to use Neuron Lang to:
* Define neurons and phenotypes.
* Export all defined neurons.
* Use `%scig` magic to search for existing ontology identifiers
* Use `LocalNameManager` to create abbreviations for phenotypes.
* Bind local names in the current python namespace using `with` or `setLocalNames`.
* Creat a phenotype context in which to define neurons using `with` or `setLocalContext`.

Please see [the documentation](https://github.com/tgbugs/pyontutils/blob/master/docs/neurons_notebook.md) in order to set up a working
environment for this notebook.

## Setup for any file defining neurons

In [1]:
from pyontutils.neurons.lang import *
# set predicates in the event that the default config options do not work
# if you cloned the NIF-Ontology into a nonstandard location change ontology_local_repo in devconfig.yaml
pred = config(out_graph_path='neuron_lang_example_neurons.ttl')
from pyontutils import phenotype_namespaces as phns  # import after config() in case config defaults fail

# WARNING only call config once at the start of your program
# if you call it again it will reset the state and can be very confusing
# see neurons.lang.config and neurons.graphBase.configGraphIO for details

In [2]:
# you can ignore this cell
# some utility functions needed for this tutorial
# due to the potential for notebooks to run cells out of order
def cellguard(addns=False):
    # unfortunately ipy.hooks['pre_run_code_hook'].add(__cellguard)
    # causes this to be called too frequently :/
    setLocalNames()
    setLocalContext()
    if addns:
        setLocalNames(phns.BBP)

## Neurons
Neuron instances are build out of Phenotype instances.
Phenotypes are object-predicate pairs that take curied
string representations (or uris) as arguments.

Note: `pred` is a helper class defined in [neurons/lang.py](https://github.com/tgbugs/pyontutils/blob/e1f5250b216c7755d1a050790ec0feaaec142aba/pyontutils/neurons/lang.py#L126) and populated [using this function in neurons.py](https://github.com/tgbugs/pyontutils/blob/e1f5250b216c7755d1a050790ec0feaaec142aba/pyontutils/neurons/core.py#L59).

In [3]:
myFirstNeuron = Neuron(Phenotype('NCBITaxon:10090'),
                       Phenotype('UBERON:0000955'))

# NOTE: label is cosmetic and will be overwritten by rdfs:label

myPhenotype = Phenotype('NCBITaxon:9685',           # object
                        pred.hasInstanceInSpecies,  # predicate (optional)
                        label='Cat')                # label for human readability

In [4]:
# str and repr produce different results
print(myFirstNeuron)

Neuron(Phenotype('NCBITaxon:10090',
                 'ilxtr:hasInstanceInSpecies',
                 label='Mus musculus'),
       Phenotype('UBERON:0000955',
                 'ilxtr:hasSomaLocatedIn',
                 label='brain'))


In [5]:
print(repr(myFirstNeuron))  # NOTE: this is equivalent to typing `myFirstNeuron` and running the cell

Neuron(Phenotype('NCBITaxon:10090', 'ilxtr:hasInstanceInSpecies', label='Mus musculus'), Phenotype('UBERON:0000955', 'ilxtr:hasSomaLocatedIn', label='brain'))


## Viewing and saving
Neuron Lang can only be used to add new neurons to a graph.
Therefore if you need to remove neruons you need to reset
the whole program. For this reason I do not suggest using
ipython notebooks since they persist state in ways that can
be very confusing when working with a persistent datastore.

In [6]:
# view the turtle (ttl) serialization of all neurons
turtle = graphBase.ttl()
print(turtle)

@prefix : <file:///ERROR/EMPTY/PREFIX/BANNED/> .
@prefix ilxtr: <http://uri.interlex.org/tgbugs/uris/readable/> .
@prefix NCBITaxon: <http://purl.obolibrary.org/obo/NCBITaxon_> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix SAO: <http://uri.neuinfo.org/nif/nifstd/sao> .
@prefix TEMP: <http://uri.interlex.org/temp/uris/> .
@prefix UBERON: <http://purl.obolibrary.org/obo/UBERON_> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<file://neuron_lang_example_neurons.ttl> a owl:Ontology ;
    owl:imports </home/tom/git/NIF-Ontology/ttl/phenotype-core.ttl> ;
    prov:wasGeneratedBy <https://github.com/tgbugs/pyontutils/blob/e62221a68a175b34d09e0177647580d9c70bd7ec/core.py#L270> .

### Classes

TEMP:0-p0-NCBITaxon-10090-0-p3-UBERON-0000955 a owl:Class ;
    owl:equi

In [7]:
# view the python serialization of all neurons
python = graphBase.python()
print(python)

#!/usr/bin/env python3.6
from pyontutils.neurons.lang import *

Config('neuron_lang_example_neurons')

# Mus musculus brain neuron
Neuron(Phenotype('NCBITaxon:10090',
                 'ilxtr:hasInstanceInSpecies',
                 label='Mus musculus'),
       Phenotype('UBERON:0000955',
                 'ilxtr:hasSomaLocatedIn',
                 label='brain'))


In [8]:
# write the turtle file defined in cell 1
graphBase.write()

In [9]:
# write a python file that has the same name as the file in cell 1 but with a .py extension
graphBase.write_python()

In [10]:
# view a list of all defined neurons

graphBase.neurons()

# NOTE: you can also use Neurons.neurons() or other subclasses
# of graphBase to query instead. The functionality is the same
# as calling graphBase.neurons() but graphBase holds the global
# state for converting classes to turtle and manages serialization.

[Neuron(Phenotype('NCBITaxon:10090', 'ilxtr:hasInstanceInSpecies', label='Mus musculus'), Phenotype('UBERON:0000955', 'ilxtr:hasSomaLocatedIn', label='brain'))]

## scig
When creating neurons we want to be able to find relevant identifiers  
quickly while working. There is a cli utility called scig that can be  
used as a cell magic `%scig` to search a SciGraph instance for terms.

In [11]:
%scig --help

Look look up ontology terms on the command line.

Usage:
    scig v [--local --verbose --key=KEY] <id>...
    scig i [--local --verbose --key=KEY] <id>...
    scig t [--local --verbose --limit=LIMIT --key=KEY] <term>...
    scig s [--local --verbose --limit=LIMIT --key=KEY] <term>...
    scig g [--local --verbose --rt=RELTYPE --key=KEY] <id>...
    scig e [--local --verbose --key=KEY] <p> <s> <o>
    scig c [--local --verbose --key=KEY]
    scig cy [--limit=LIMIT] <query>
    scig onts [options]

Options:
    -l --local          hit the local scigraph server
    -v --verbose        print the full uri
    -t --limit=LIMIT    limit number of results [default: 10]
    -k --key=KEY        api key
    -w --warn           warn on errors


In [12]:
# use -t to limit the number of results
%scig -t 1 t hippocampus -v

hippocampus
http://localhost:9000/scigraph/vocabulary/term/hippocampus?limit=1
	BIRNLEX:721
		abbreviations: []
		acronyms: []
		categories: ['anatomical entity']
		definitions: ['A part of the brain consisting of a three layered cortex located in the forebrain bordering the medial surface of the lateral ventricle. The term hippocampus is often used synonymously with hippocampal formation which consists of the hippocampus proper or Cornu Ammonis, the dentate gyrus and the subiculum.']
		deprecated: True
		iri: http://uri.neuinfo.org/nif/nifstd/birnlex_721
		labels: ['Hippocampus']
		synonyms: ['ammon horn', 'Cornu ammonis', 'hippocampus proper', "Ammon's horn"]



In [13]:
# you can escape spaces with \
%scig t macaca\ mulatta

macaca mulatta
	NCBITaxon:9544
		abbreviations: []
		acronyms: []
		categories: []
		definitions: []
		deprecated: False
		iri: http://purl.obolibrary.org/obo/NCBITaxon_9544
		labels: ['Macaca mulatta']
		synonyms: ['Rhesus monkey', 'rhesus macaque', 'rhesus monkeys', 'rhesus macaques']



In [14]:
# quotes also allow search with spaces
%scig -t 1 s 'nucleus basalis of meynert'

nucleus basalis of meynert
	SCR:003348
		abbreviations: ['PIN']
		acronyms: []
		categories: ['Resource']
		definitions: ['THIS RESOURCE IS NO LONGER IN SERVCE, documented September 2, 2016. Relational database containing the compositions of multi-protein complexes in the nucleus of budding yeast and human cells. Its content is limited to information curated from the proteomics literature and primarily comprises of components of the general transcription and DNA repair machinery. In addition to database browsing and searching capabilities, the PINdb web portal also includes user-friendly interactive tools for comparative analysis of the composition of multiple protein complexes and for clustering and visualizing network of protein complexes. Currently, PINdb contains mostly protein complexes that may be involved in gene transcription. To facilitate comparative analyses and identification of protein complexes, the compositional information is integrated with standardized gene nomenclatu

In [15]:
# without quotes scig will search multiple terms at once
%scig -t 1 t cat mouse

cat
	SCR:007238
		abbreviations: []
		acronyms: []
		categories: ['Resource']
		definitions: ['CaTS is a simple, multi-platform interface for carrying out power calculations for large genetic association studies, including two stage genome wide association studies. Sponsors: This research was supported by the US National Institutes of Health. Keywors: Calculator, Interface, Genetic, Association, Genome, Study,']
		deprecated: False
		iri: http://scicrunch.org/resolver/SCR_007238
		labels: ['Power Calculator for Two Stage Association Studies']
		synonyms: ['CaTS']

mouse
	NCBITaxon:10090
		abbreviations: []
		acronyms: []
		categories: ['organism']
		definitions: []
		deprecated: False
		iri: http://purl.obolibrary.org/obo/NCBITaxon_10090
		labels: ['Mus musculus']
		synonyms: ['mouse', 'house mouse', 'mice C57BL/6xCBA/CaJ hybrid', 'Mus muscaris']



## Accessing SciGraph directly from python

In [16]:
from pyontutils.scigraph_client import Graph, Vocabulary

sgg = Graph()
sgv = Vocabulary()

terms = sgv.findByTerm('neocortex')
nodes_edges = sgg.getNeighbors('UBERON:0000955', 
                               relationshipType='BFO:0000050',  # part of
                               direction='INCOMING')
print('synonyms:', terms[0]['synonyms'])
print('subjects:', *(e['sub'] for e in nodes_edges['edges']))

synonyms: ['iso-cortex', 'homotypical cortex', 'nonolfactory cortex', 'neocortex (isocortex)', 'cerebral neocortex', 'isocortex (sensu lato)', 'nucleus hypoglossalis', 'isocortex', 'neopallial cortex', 'homogenetic cortex', 'neopallium']
subjects: UBERON:6003626 UBERON:0002616 UBERON:0003544 UBERON:0005282 UBERON:0001058 UBERON:0003528 UBERON:0013694 UBERON:0005838 UBERON:0022776 UBERON:0003547 UBERON:0008998 UBERON:0007702 UBERON:0003052 UBERON:0003053 UBERON:0006796 UBERON:0000315 UBERON:0013146 UBERON:0017631 UBERON:0003947 UBERON:0005075 UBERON:0010009 UBERON:0006795 UBERON:0010403 UBERON:0001059 UBERON:0013118 UBERON:0000454


## Namespaces - context managers
We can be more concise by creating a namespace for our phenotype names.  
Normally these are defined in another file (e.g. [phenotype_namespaces.py](https://github.com/tgbugs/pyontutils/blob/master/pyontutils/phenotype_namespaces.py)) so that they can be shared and reused.

NOTE: for a full explication of phenotype namespaces see [neurons/example.py](https://github.com/tgbugs/pyontutils/blob/master/pyontutils/neurons/example.py#L20)


In [17]:
from pyontutils.neurons import LocalNameManager
from pyontutils.utils import TermColors as tc  # pretty printing that is not part of this tutorial

class myPhenotypeNames(LocalNameManager):  # see neurons.LocalNameManager
    Mouse = Phenotype('NCBITaxon:10090', pred.hasInstanceInSpecies)
    Rat = Phenotype('NCBITaxon:10116', pred.hasInstanceInSpecies)
    brain = Phenotype('UBERON:0000955', pred.hasSomaLocatedIn)
    PV = Phenotype('PR:000013502', pred.hasExpressionPhenotype)
    
# you can see all the mappings in a local name manager by printing it or repring it
print(myPhenotypeNames)

# with a context manager we can use a namespace to create neurons
# more concisely and more importantly to repr them more concisely

with myPhenotypeNames:
    n = Neuron(Rat, brain, PV)
    
    # printing is unaffected so the fully expanded form is always
    # accessible (__str__ vs __repr__)
    print(tc.red('print inside unchanged:'), n, sep='\n')
    
    print(tc.red('repr inside inside:'), repr(n))
    
    # we can also repr a neuron defined elsewhere using our own names
    
    print(tc.red('repr outside inside:'), repr(myFirstNeuron))

# outside the context manager our concise repr is gone
print(tc.red('repr inside outside:'), repr(n))

# in addition we will now get a NameError of we try to use bare words
try: Neuron(Rat)
except NameError: print(tc.blue('Rat fails as expected.'))

class myPhenotypeNames(LocalNameManager):
    Mouse    = Phenotype('NCBITaxon:10090', 'ilxtr:hasInstanceInSpecies', label='Mus musculus')
    PV       = Phenotype('PR:000013502', 'ilxtr:hasExpressionPhenotype', label='parvalbumin alpha')
    Rat      = Phenotype('NCBITaxon:10116', 'ilxtr:hasInstanceInSpecies', label='Rattus norvegicus')
    brain    = Phenotype('UBERON:0000955', 'ilxtr:hasSomaLocatedIn', label='brain')
print inside unchanged:
Neuron(Phenotype('NCBITaxon:10116',
                 'ilxtr:hasInstanceInSpecies',
                 label='Rattus norvegicus'),
       Phenotype('UBERON:0000955',
                 'ilxtr:hasSomaLocatedIn',
                 label='brain'),
       Phenotype('PR:000013502',
                 'ilxtr:hasExpressionPhenotype',
                 label='parvalbumin alpha'))
repr inside inside: Neuron(Rat, brain, PV)
repr outside inside: Neuron(Mouse, brain)
repr inside outside: Neuron(Phenotype('NCBITaxon:10116', 'ilxtr:hasInstanceInSpecies', label='Rattus n

## Namespaces 2 - global modification

In [18]:
cellguard()

# there are already many namespaces defined in phenotype_namespaces.py
print(tc.red('Namespaces:'), phns.__all__)

# setLocalNames adds any names from a namespace to the current namespace
setLocalNames(phns.Species)

# we can load additional names
setLocalNames(phns.Regions, phns.Layers)

# however we will get a ValueError on a conflict
try:
    setLocalNames(phns.Test)
except ValueError as e:
    print(tc.red('The error:'), e)

# we can extend namespaces as well (again, best in a separate file)
# as long as the local names match we can combine entries
class MoreSpecies(phns.Species, myPhenotypeNames):
    Cat = myPhenotype
    ACh = Phenotype('CHEBI:15355', pred.hasExpressionPhenotype)
    AChMinus = NegPhenotype(ACh)
    
with MoreSpecies:
    can = Neuron(Cat, ACh, L2)
    cant = Neuron(Cat, AChMinus, L3)
    print(tc.red('More species:'), can, cant, sep='\n')

# we can also refer to phenotypes in a namespace directly
n = Neuron(Mouse, MoreSpecies.ACh)
print(tc.red('Direct usage:'), n, sep='\n')

# getLocalNames can be used to inspect the current set of defined names
print(tc.red('getLocalNames:'), sorted(getLocalNames().keys()))

# clear the local names by calling setLocalNames with no arguments
setLocalNames()

# no more short names ;_;
try: Neuron(Mouse, PV)
except NameError: print(tc.blue('Neuron(Mouse, PV) fails as expected'))

# for the rest of these examples we will use the BBP namespace
setLocalNames(phns.BBP)


# define neurons using our local names
Neuron(Mouse, L23, CCK, NPY)
Neuron(Mouse, brain, L3, PV)
Neuron(PV, DA)

cellguard()

Namespaces: ['Test', 'Layers', 'PaxRatLayers', 'Regions', 'PaxRatRegions', 'Species', 'BBP']
The error: Mapping between LocalNames and phenotypes must be injective.
Cannot cannot bind 'LOOK_AT_THE_CUTE_LITTLE_GUY' to Phenotype('NCBITaxon:10116', 'ilxtr:hasInstanceInSpecies', label='Rattus norvegicus').
It is already bound to 'Rat'
More species:
Neuron(Phenotype('NCBITaxon:9685',
                 'ilxtr:hasInstanceInSpecies',
                 label='Felis catus'),
       Phenotype('UBERON:0005391',
                 'ilxtr:hasLayerLocationPhenotype',
                 label='cortical layer II'),
       Phenotype('CHEBI:15355',
                 'ilxtr:hasExpressionPhenotype',
                 label='acetylcholine'))
Neuron(Phenotype('NCBITaxon:9685',
                 'ilxtr:hasInstanceInSpecies',
                 label='Felis catus'),
       Phenotype('UBERON:0005392',
                 'ilxtr:hasLayerLocationPhenotype',
                 label='cortical layer III'),
       NegPhenotype('CHE

## Context - context managers

In [19]:
cellguard(True)

# we often want to create many neurons in the same contex
# the easiest way to do this is to use a instance of a neuron
# as the input to a context manager
with Neuron(Rat, CA1):
    n1 = Neuron(CCK)
    n2 = Neuron(NPY)
    n3 = Neuron(PC)

# neurons always retain the context they were created in
print(tc.red('example 1:'), *map(repr, (n1, n2, n3)), '', sep='\n')

# you cannot change a neuron's context but you can see its original context
print(tc.red('example 2:'), n3.context, '', sep='\n')
try:
    n3.context = Neuron(Mouse, CA2)
except TypeError as e:
    print(tc.red('error when setting context:'), e, '\n')

# you can also use with as syntax when creating a context
with Neuron(Mouse) as n4:
    n5 = Neuron(CCK)

print(tc.red('example 3:'), *map(repr, (n4, n5)), '', sep='\n')

# contexts cannot violate disjointness axioms
try:
    with Neuron(Rat):
        print(tc.red('neuron ok:'), Neuron(), '', sep='\n')
        with Neuron(Mouse):
            print('This will not print')
except TypeError: print(tc.blue('Neuron(Rat, Mouse) fails as expected\n'))

# if you define a new neuron inside a context it will carry
# that context with it if used to define a new context

# context does not nest for neurons defined outside a with

with n3:
    n6 = Neuron(VIP)
    with n5:                 # defined outside does not nest
        n7 = Neuron(SOM)
    with Neuron(SLM) as n8:  # defined inside nests
        n9 = Neuron(SOM)
    n10 = Neuron(SOM)

print(tc.red('example 4:'), *map(repr, (n3, n6, n5, n7, n8, n9, n10)), sep='\n')

# 
with Neuron(Rat), Neuron(CTX) as context:
    print(context)
    n11 = Neuron(L1)
    print(n11)

cellguard()

example 1:
Neuron(Rat, CA1, CCK)
Neuron(Rat, CA1, NPY)
Neuron(Rat, CA1, PC)

example 2:
(Phenotype('NCBITaxon:10116', 'ilxtr:hasInstanceInSpecies', label='Rattus norvegicus'), Phenotype('PAXRAT:322', 'ilxtr:hasSomaLocatedIn', label='field CA1 of the hippocampus (paxrat)'))

error when setting context: Cannot change the context of an instantiated neuron. 

example 3:
Neuron(Mouse)
Neuron(Mouse, CCK)

neuron ok:
Neuron(Phenotype('NCBITaxon:10116',
                 'ilxtr:hasInstanceInSpecies',
                 label='Rattus norvegicus'))

Neuron(Rat, Mouse) fails as expected

example 4:
Neuron(Rat, CA1, PC)
Neuron(Rat, CA1, PC, VIP)
Neuron(Mouse, CCK)
Neuron(Mouse, CCK, SOM)
Neuron(Rat, CA1, SLM, PC)
Neuron(Rat, CA1, SLM, PC, SOM)
Neuron(Rat, CA1, PC, SOM)
Neuron(Phenotype('NCBITaxon:10116',
                 'ilxtr:hasInstanceInSpecies',
                 label='Rattus norvegicus'),
       Phenotype('UBERON:0001950',
                 'ilxtr:hasSomaLocatedIn',
                 label='neoco

## Context 2 - global modification

In [20]:
cellguard(True)

# like namespaces you can also set a persistent local context
context0 = Neuron(CCK, NPY, SOM, DA, CA1, SPy)
context1 = Neuron(Rat, S1, L4)
setLocalContext(context0)
print(tc.red('created with context:'), repr(Neuron(TPC)))

# contexts are addative
# to change context using a Neuron you need to setLocalContext() first

# without resetting we get a disjointness error
try: setLocalContext(Neuron(Rat, S1, L4))
except TypeError as e: print(tc.blue('Neuron(S1, CA1) fails as expected'), e)
    
# reset
setLocalContext()

# now we will not get an error
setLocalContext(Neuron(Rat, S1, L4))
print(tc.red('Success:'), repr(Neuron(PC)))

# a neuron declared in a different context can be used to change the context withour resetting
# if you know in advance that you will be dealing with multiple contexts, I suggest you
# create all of those context neurons first so that they are available when needed
setLocalContext(context0)

# like namespaces call getLocalContext to see the current context
print(tc.red('getLocalContext:'), *(p.pShortName for p in getLocalContext()))

# like namespaces calling setLocalContext without arguments clears context
setLocalContext()
print(tc.red('no context:'), repr(Neuron(brain)))

cellguard()

created with context: Neuron(CA1, SPy, TPC, DA, CCK, NPY, SOM)
Neuron(S1, CA1) fails as expected Disjointness violated for http://uri.interlex.org/tgbugs/uris/readable/hasSomaLocatedIn due to [Phenotype('PAXRAT:322', 'ilxtr:hasSomaLocatedIn', label='field CA1 of the hippocampus (paxrat)'), Phenotype('PAXRAT:794', 'ilxtr:hasSomaLocatedIn', label='primary somatosensory cortex (paxrat)')]
Success: Neuron(Rat, S1, L4, PC)
getLocalContext: CA1 SPy DA CCK NPY SOM
no context: Neuron(brain)


## Context 3 - the old way

In [21]:
cellguard(True)

context = (Rat, S1)
ca1_context = (Rat, CA1)

def NeuronC(*args, **kwargs):
    return Neuron(*args, *context, **kwargs)

def NeuronH(*args, **kwargs):
    return Neuron(*args, *ca1_context, **kwargs)

neurons = {
    'HBP_CELL:0000013': NeuronC(CCK),
    'HBP_CELL:0000016': NeuronC(PV),
    'HBP_CELL:0000018': NeuronC(PC),
    'HBP_CELL:0000135': NeuronH(SLM, PPA),
    'HBP_CELL:0000136': NeuronH(SO, BP),
    'HBP_CELL:0000137': NeuronH(SPy, BS),
    'HBP_CELL:0000148': Neuron(Rat, STRI, MSN, D1),
    'HBP_CELL:0000149': Neuron(Rat, CA3, PC),
        }
neurons['HBP_CELL:0000013']

cellguard()

## Disjointness
Neuron Lang enforces basic disjointness on phenotypes of 'data' level neurons


In [22]:
cellguard(True)

try: Neuron(Mouse, Rat)
except TypeError as e: print(tc.blue('Neuron(Mouse, Rat) fails as expected'), e, sep='\n')
    
cellguard()

Neuron(Mouse, Rat) fails as expected
Disjointness violated for http://uri.interlex.org/tgbugs/uris/readable/hasInstanceInSpecies due to [Phenotype('NCBITaxon:10090', 'ilxtr:hasInstanceInSpecies', label='Mus musculus'), Phenotype('NCBITaxon:10116', 'ilxtr:hasInstanceInSpecies', label='Rattus norvegicus')]
